In [2]:
samples = [
    ("(10 + 4 * 2 + 1)", 29, 14 * 3),
    ("((10 + 4) * 2 + 1)", 29, 14 * 3),
    ("((10 + 4 * 2) + 1)", 29, 29),
    ("10 + 4", 14, 14),
    ("1 + 2 * 3 + 4 * 5 + 6", 71, 3 * 7 * 11),
    ("2 * 3 + (4 * 5)", 26, 2 * (3 + 4*5)),
    ("5 + (8 * 3 + 9 + 3 * 4 * 3)", 437, 5 + 8 * (3 + 9 + 3) * 12),
    ("5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))", 12240, 5 * 9 * 7*3*12*(3+(14*4))),
    ("((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2", 13632, (6*9*(15*14 +6)+6)*2),
]


In [3]:
def parse_token(expr, ini):
    not_numbers = r"(+*)"
    while expr[ini] == ' ':
        ini += 1
    if expr[ini] in not_numbers:
        return expr[ini], ini + 1
    # Otherwise it is a number...
    end = ini + 1
    while end < len(expr) and expr[end] not in not_numbers:
        end += 1
    return int(expr[ini:end]), end

def parse_expr(expr, ini=0):
    result = []
    while ini < len(expr):
        token, ini = parse_token(expr, ini)
        result.append(token)
    return result
assert parse_expr('10 + 4') == [10, '+', 4]
assert parse_expr('10 + ()4*5') == [10, '+', '(', ')', 4, '*', 5]

def eval_tree(tree):
    if isinstance(tree, int):
        return tree
    if len(tree) == 1:
        return eval_tree(tree[0])
    lt, op, rt = tree
    left = eval_tree(lt)
    right = eval_tree(rt)
    if op == '*':
        return left * right
    if op == '+':
        return left + right
    assert False, tree

def invert_tree(tree):
    # print(tree)
    result = _invert_tree_aux(tree, None, None)
    # print(result)
    return result

def _invert_tree_aux(tree, left_tree, left_op):
    assert not isinstance(left_tree, list), left_tree
    if isinstance(tree, int):
        if left_tree:
            return (left_tree, left_op, tree)
        else:
            return tree
    if len(tree) == 1:
        # Parenthesis
        #print('Parenth', tree)
        rt = _invert_tree_aux(tree[0], None, None)
        #print(tree, '->', rt)
        assert not isinstance(rt, list), rt
        if left_tree:
            return (left_tree, left_op, rt)
        else:
            return rt
    lt, op, rt = tree
    nlt = _invert_tree_aux(lt, None, None)
    if left_tree:
        assert not isinstance(nlt, list), nlt
        nlt = (left_tree, left_op, nlt)
    #print('to invert:', rt, nlt, op)
    return _invert_tree_aux(rt, nlt, op)

def eval_expr(expr):
    tokens = parse_expr(expr)
    tree = parse_tokens(tokens)
    result = eval_tree(invert_tree(tree))
    # print('Problem1', result)
    return result

def parse_tokens(tokens):
    tree, _ = parse_tokens_expr(tokens, 0)
    return tree

def parse_tokens_expr(tokens, start):
    tree_left, start = parse_tokens_value(tokens, start)
    if start == len(tokens) or tokens[start] == ')':
        return tree_left, start + 1
    op = tokens[start]
    tree_right, start = parse_tokens_expr(tokens, start + 1)
    return (tree_left, op, tree_right), start

def parse_tokens_value(tokens, start):
    if tokens[start] == '(':
        tree, start = parse_tokens_expr(tokens, start + 1)
        return [tree], start
    assert isinstance(tokens[start], int), tokens[start]
    return tokens[start], start + 1

for sample in samples:
    print('Chjeck', sample)
    assert eval_expr(sample[0]) == sample[1], sample
print('OK')
'OK'

sum([eval_expr(x) for x in my_in.split('\n') if x])

Chjeck ('(10 + 4 * 2 + 1)', 29, 42)
Chjeck ('((10 + 4) * 2 + 1)', 29, 42)
Chjeck ('((10 + 4 * 2) + 1)', 29, 29)
Chjeck ('10 + 4', 14, 14)
Chjeck ('1 + 2 * 3 + 4 * 5 + 6', 71, 231)
Chjeck ('2 * 3 + (4 * 5)', 26, 46)
Chjeck ('5 + (8 * 3 + 9 + 3 * 4 * 3)', 437, 1445)
Chjeck ('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))', 12240, 669060)
Chjeck ('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2', 13632, 23340)
OK


NameError: name 'my_in' is not defined

In [4]:

def eval_expr2(expr):
    tokens = parse_expr(expr)
    tree = parse_with_prio(tokens)
    # print(tree)
    result = eval_tree(tree)
    # print('Problem2', result)
    return result

def parse_with_prio(tokens, start=0):
    terms = []
    place = start
    while place < len(tokens):
        if tokens[place] == '(':
            subtree, place = parse_parenth(tokens, place)
            assert tokens[place] == ')'
            terms.append(subtree)
            place += 1
            continue
        if tokens[place] == ')':
            return terms, place
        terms.append(tokens[place])
        place += 1

        
    return parse_terms(terms)

def parse_terms(terms):
    ini = 0
    end = 0
    factors = []
    while end < len(terms):
        if terms[end] != '*':
            end += 1
            continue
        assert terms[end] == '*'
        factors.append(terms[ini:end])
        end += 1
        ini = end
    if end > ini:
        factors.append(terms[ini:end])
    tree = parse_factors(factors)
    return tree

def parse_factors(factors):
    tree = None
    for f in factors:
        ff = parse_factor(f)
        if tree:
            tree = (tree, '*', ff)
        else:
            tree = ff
    return tree

def parse_factor(factor):
    t = factor[0]
    assert isinstance(t,(int, tuple))
    if len(factor) == 1:
        return t
    assert factor[1] == '+'
    return (t, '+', parse_factor(factor[2:]))
    
    
assert parse_factors([[3],[4]]) == (3, '*', 4), parse_factors([[3], [4]])

def parse_parenth(tokens, start):
    assert tokens[start] == '('
    terms, place = parse_with_prio(tokens, start + 1)
    tree = parse_terms(terms)
    assert tokens[place] == ')'
    return tree, place

_aux = parse_expr('1 + ( 3 * 4 ) * 2 + 5')
assert _aux == [1, '+', '(', 3, '*', 4, ')', '*', 2, '+', 5], _aux
_got = parse_with_prio(_aux)
_expected = ((1, '+', (3, '*', 4)), '*', (2, '+', 5))
assert _got == _expected, _got

for sample in samples:
    print('Chjeck', sample)
    assert eval_expr2(sample[0]) == sample[2], sample
print('OK')
'OK'
sum([eval_expr2(x) for x in my_in.split('\n') if x])

Chjeck ('(10 + 4 * 2 + 1)', 29, 42)
Chjeck ('((10 + 4) * 2 + 1)', 29, 42)
Chjeck ('((10 + 4 * 2) + 1)', 29, 29)
Chjeck ('10 + 4', 14, 14)
Chjeck ('1 + 2 * 3 + 4 * 5 + 6', 71, 231)
Chjeck ('2 * 3 + (4 * 5)', 26, 46)
Chjeck ('5 + (8 * 3 + 9 + 3 * 4 * 3)', 437, 1445)
Chjeck ('5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))', 12240, 669060)
Chjeck ('((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2', 13632, 23340)
OK


NameError: name 'my_in' is not defined

In [5]:
my_in = """7 * (7 + (4 + 5) + 4)
2 * 6 + 7 * 3
5 * 5 + 8 * ((7 * 4 + 7 * 5 * 4) * 8 + 2 + 2 + 6 + 9) + 8 + 2
8 * 6 * 7 * 5 + ((5 * 7 + 5 + 4 * 3 * 5) * 7 + (3 + 3 + 3 * 2) * 7 + (4 * 3 * 4) + 3) * 9
7 * 4 * 8 + (2 + 5 * (8 + 9) * 7 * (9 + 8 + 4) + 8)
((2 * 9 + 2 * 7 * 3) + 8 * 3 * 5 * 6) + 5
6 + 5 * 6 * (9 * 9 * 8 + (5 + 5 * 3 + 8 * 3 + 7) * 6 + 6) + 5 + (9 * 8 + (2 + 3 + 2 + 2 * 3 * 9) + 7 * (8 + 3 * 3 + 3 + 5 * 7))
(5 * 4 * (5 * 9 + 6 * 8 + 4 * 3)) * 7 * 5 * 7 * (3 + 7 + 3 * 2 + 6) * (3 + 3)
((8 * 8 * 9 + 4 * 5 + 2) + 8) + 3
2 * 6 * 9 + (7 + 2 + 5 + (8 * 3)) * (9 * 6 * 9 * 9) * 5
9 + 6 + 5 * 2
4 + (5 * 6 + 6) + (3 * 3 + 8 + 8) + (5 + 4) + (9 * 8 + (6 * 4 * 2) + 5) * 6
4 + (2 + 6 * 4 + (7 + 9 * 5) * 3 + 6) * 4
6 * 9
((5 + 3) * 8 + 5 + 6) + 6
4 + 9 * 3 + 8 + 3 * (3 * (8 * 5 + 5 * 7 * 5 + 2) * 3 + 6 * (4 + 6 + 8) + 7)
8 + (4 + 2 * 2 * 2 + 5) + 2 + (7 * (7 + 7 + 7 + 7) + 7 + 2) * ((8 + 9 * 6 + 6 * 5) + 6 + 7 * 4 * (3 + 8 * 6 + 5 + 3 * 6) * 7) * 5
(4 * 7 + 4 + 6 + (6 * 5) * 6) * 6 * 8
4 * 3 + 9 + 2 * 4 * 2
5 + (4 * 3 + 8 + 6 * 7 + 9) * 2
4 * (3 + 3 + 6 + 4)
5 + (9 + (9 + 9 * 8 + 8) * 3) + 6 + 5 * 2
(8 * (3 + 2 * 5 + 5 * 4) * 6 * 3) + (2 * 8) * 6 * (3 * 8) + 9 * 4
5 * 4 + (2 * 7 + 7 * (4 + 3 * 3 + 5) + 9)
(4 * 6 * 5 + 5 * 3) * 2 * 5 * 6 * 4 + 9
6 * 8 + (7 + 4) + 5 * (3 * 9)
2 * 7 + 9 * 5 * ((7 + 4 + 2 * 6) * 3 + 5 + 4 + (8 * 9 * 4 + 3 + 7)) + 9
9 + 7 * (6 * 7 * 9 * 9 + 2) * (2 + 8 + 8) * 3 + 7
(8 * 8 * 3 * (9 * 8 + 3 + 2 + 3 * 5) + 4) + 3 + 2 + (8 + 6)
8 + 5 + 3 + 8 * ((2 * 8 + 7 + 7 * 4) + 7) + 8
9 * 3 * 2 + 6
2 * 9 + (9 * 5) * 5 + 3
4 * 7 * (7 * 6 * 8 + 8) * 6 * 4 + (8 * 2 * 5 * 4 * 5 * 3)
((6 + 9 + 9 * 4 * 2 + 5) + 8 + 6 * 4) * 3 + 7 * 2 + 9
3 * 7 + 9 + 7 * (5 * 2 * (6 * 9 * 6) + 9) * 4
3 + 5 + 3 * (6 * (4 * 2 + 9 * 7 * 4) * 7) + 5
2 + 6 + 2 * 7 + (9 + 2 * 4 * (9 * 3) + 8)
(9 * (9 * 7 + 2) * 4 * 7 * (9 * 4)) * ((2 * 9) + 6 + (7 * 7 * 4 * 4) * 4 + 5)
(2 + (8 + 8 * 7 * 2) * 7 * 3 * 7 * 6) + 3 + (7 + 6 + 4) + 4 + 4 + 9
((6 + 5 + 4 * 7) + 3 + 6) + 2 * 3 + 8 + 2 * 3
(4 * (8 * 7 * 7) * (4 * 4 + 9 * 4) + (5 + 4 + 4 * 9 + 9) * (4 * 9 + 2 + 2)) * 2
((2 * 3 * 8 * 3) * 4 + 8 * (2 + 6) + 9) + 5 * 7 + 6 + ((5 + 9) + 4)
8 + (4 * 8 + 5 + 7 * 9 + 4) * 9 + (6 * 7 * 2 + 7)
(2 * 5 * 9 * 2 + 5) + ((2 + 9) + 2 * (2 * 6 + 7)) + 6 + 8 * 5
4 + (4 * 6) + 7 * 3 + 6 + 3
(3 + 4 * 4 + 3 + 4) + 2 * 2
9 * 3 + 5 * (3 + 5 * 9) + 5 + (6 + 7)
6 * ((9 + 9 + 2 + 3 * 9 + 8) + 3 + 7 * (9 * 6 + 8 + 8)) + (8 * (6 * 2 * 6 + 9 * 5) + 8 * (9 * 3 * 4 * 9 + 2 + 9) * (3 * 7 + 2 * 8 * 2 + 9))
((9 * 3 * 4 * 9) + 7 * 3 + (6 * 8 * 6 * 5 * 6) + 8 * 3) + 4
9 * 8 + 5 + 3 + (9 * 8 * 3 + 4)
(8 * (9 + 5 * 8) + 9) + 5 + 4 * 7 + 6 * (6 + 7 + 4 * 2 + 8 + 8)
(8 * (7 + 3 * 9 * 5 * 6 + 8) * (6 * 9 * 3) * (3 * 8) * 2) + 4 + 7 * 7 * 8 * 8
(8 * 9) * 6 * 8
((4 * 9 * 5) * 3 * 9) * 7
(9 + 4) * (5 * (9 * 4 * 5 * 2 * 2) * 2 * (4 + 2) + 6 + 6) * (8 + 9 + 5 + 6)
(3 * 4) * (8 * 3) + 5 * ((9 * 7 * 9 + 2) * 9 + (9 + 8) * 4 * (6 * 5 + 2 + 3 + 8 * 4)) * 4 + 8
9 * (2 * 5 * 3 + (2 * 9 * 8 * 2 * 2) + (2 * 6 * 2) * 5) * 3 + 5
9 + 4 * ((4 * 8 + 2 + 7 * 5) * 6)
9 + 3 * (5 * 4 + 7 + 9) + (4 * (4 * 8 * 8) * 7) + 9
(2 * 5 * 3 + 7) * 6 + 3 * 8 + (7 + 2 * 2) * 8
5 * 7 * (6 * 4 + 7 + 3 + 5 * 3) * (8 + 8 + (5 * 8 * 3 * 7 + 3) * 3 + (3 + 9 * 5) + 7) + (9 * (3 * 9 + 3 * 3) * 6 * 7 * 3) * 9
(6 * 2 + (9 * 7 + 6 + 8 + 7 + 6) * 8 * 8 + 2) + 3
4 + ((8 * 8 + 6 + 6) + (5 * 9 * 5 + 5) + (6 + 3 * 4) * 4)
4 + (9 * 4 + 6 + 7 + 5) * 2 * (4 + 7)
7 + 8 * 3 + (8 * (9 + 9 * 9 + 9 * 8 + 4)) + (4 + 9) * ((9 * 7) + 2)
4 * 4
(4 * (9 * 9) * 2 * 8 * 6) + 4 * (7 + 4 * 3 * 9)
(9 * 6) * 5 + ((2 + 4 * 2 * 5) * 5) * 2 + 5
8 * (7 * 4 * 6 + (2 + 6 + 9 + 7) * (4 + 7))
3 * 2 + ((8 * 5 + 8 * 4 * 7) * 8 + (7 + 5 * 6 * 5 + 4 * 5) + (8 + 7 + 2 + 3 * 2) + 2) + ((6 + 3 * 3) * 2 + (2 + 2 + 5 + 8 + 6) + 6 + 9) * 9 * 2
8 * (4 + (7 + 5 * 9)) + 7
5 * 5 + 7 + 2 * (6 * 3)
(4 * (9 + 6 + 5)) * 4 * 9 * 8 + 9 + 7
2 * (4 * 3 * 6)
(2 * 2 + 2 + 7 * 3 + 3) + (3 + (5 + 2 * 2 + 8 * 5))
(9 + 5) + 3 + 3 * 4
3 + 4 + 7 + (2 + 9 * 4 * 2) + 7
9 + 3 + 2 + (7 + (6 + 4 * 3) + 8 * 2 + 7 * 3)
((4 * 7 + 2 + 8 * 6 + 9) * 8) + 2 + 2 * 3 + (8 * 8 * 6 + 9)
(6 + 7 * 3) * 2 + 8 + (7 * 6 + 3)
3 + 5 + 7 * ((2 + 8 * 5 + 4 * 4 + 9) * 4 + (8 * 3 + 8 + 5 * 2) + (9 * 8 * 8 + 9 * 5) * (3 * 3 * 3 * 4 + 7))
(3 * 5 * (9 + 3 + 7 + 8 + 8 * 8) + (6 + 5 + 7 + 8 * 5) * 2 + 9) + 9
6 * 8 + 8 + (4 * (9 * 9) * 6) * 5
9 * 8 * 3 + (7 * 2 + 6)
2 * (3 * 9 * 5) * (2 + 4 + (3 * 2 * 4 * 9 * 9)) + 9 * ((3 * 6 * 4 * 6 + 5) + (9 + 2 * 5 + 5 + 5) + 9) + 5
3 * 8 + (6 * 3 * 9 + 3 * 2) + 4 * (3 * 3 + 2 + 7 + (2 + 6 + 8 * 7 * 9)) * 5
4 + (2 + 9 * 9 * 2 * 2) + 3 * 4 * 8
((6 * 6 * 5) + 4) + 9 + (6 * 3 * 4 + 6 * (6 * 6 * 4))
(6 * 7 * 9) * 4 + 8 * 6 * ((3 + 7 * 2 * 9) + 2)
6 + 5 * 2 * 6 * 2
(2 + 6 + 2 * 6 + 8) * 3 + (5 + 7 * 5) + 2 + 9
(2 + 5) * (4 + 3 + 2 * 5)
((7 * 2 + 9 + 8 * 2) + 3 + 3 + 7 * 3) * 3 + 8 * 3 * 2
9 * 9 * 5 + 7 * ((6 + 8 + 5 * 4) + 6)
7 * 2 + 3 * (2 * 2) * 8
7 * 3 + 2
7 * 4 + 7 + (3 + (9 + 3 * 4 + 5 + 8 * 9))
3 * 8 + 6 + 5 * (9 + 8 * 9 * 3 * 8) * 5
3 * (2 * 6 * (7 * 2 + 4 * 9) + (3 + 9 * 9 + 4) + 7) + 7 + (4 + (3 * 8 + 7 + 6 + 5)) * 6 + 5
(9 + 4) * 8 + 6 * 8 * 7 + 7
6 + 6 + (5 + 7 + 5) + 2 * 8 * 9
2 * (2 + 4 * (4 + 7 + 4 * 8 * 7 * 3)) + ((5 + 4 * 9) * 4) * 4 + 5 + 5
2 * 6 * 7 * (5 * 7 * (8 * 8 + 6) + 7 * (3 + 4 + 2 * 9))
((9 * 6 + 4 + 8 + 7 * 6) + 6) + 4 * 2
8 + (8 + (5 * 6 + 8 * 2 * 5 + 3) + 5) + 7 * 2 * (4 + 6 + 3 + 7) + 6
6 * 6 + 4 * 5 * (2 * (3 + 9 * 3 + 6) + (3 + 8 + 4 + 3 * 3 * 7) + 4 + (5 * 5 + 2 * 8 + 7))
(9 * 8 + 6 * 8 * 2) * 4 * 3 + 9 + 3 * (2 + 3)
6 * 2 * 9
2 * 4 * 8
9 + (5 + 3 * (5 * 6 * 8 * 7) * (7 * 3 * 7 + 2 * 2 + 7) + 9 * 2) * 6
7 * 3 + 4 * (4 * 2 * (3 + 8 + 5 * 6 + 2 + 6) * (5 + 7) + (3 + 5 * 4 + 2 + 2 + 6) * 6) * (8 + 5 * 5 + 4 * 6)
5 + (4 + 9)
8 * (3 + 9 * 5) + 3 + 7 * 3
(2 + 5 + 7 * 8 + 4) * (6 + 7) + 5 * 6 * 2
((9 + 8 + 5 * 8 * 4 + 7) * 6 + 9) * 3 * 7
4 * ((6 + 6 + 4 * 5 * 7) * 6 * 2 + 5 + 3 + 7) * 7 + 3 * 6 + 3
6 * 7 * 4 * 7 * 3
2 + 4 * 5 * 4 + (9 + (9 * 7) + 4 * 6) * 3
(2 + 9 + 7 * 3 * (5 + 5 * 7 + 3) * 4) * 6 * 2 * 9
(6 * 2 * 2 + 3 * 7) * (4 + 8 * 6 + 5 + 2) + 4 + 6
5 + (4 + 6 + (9 * 5 + 6 + 5 * 2) + 4) + 4 * 8
3 + 6 + ((4 * 4 * 6 * 7 + 6) * 5 * 3 * 7) * (3 * 8 + 6 + 8 + 5) + 8
3 * 8 + (4 + (9 * 5 * 3 * 9 + 7 + 5)) + 2
4 * 9 * 2 * 4 + 4 * 8
(6 + 2 * (5 * 5 + 2 * 3) * 5) * 7 + (2 + 2 + 2)
7 * (3 + 3 + 9) * 7
8 * 7 + (4 + 9 + 6 * 6 + 3) * (2 * 2 * (9 * 2 * 6))
8 + (6 + 2 + 9 + 2) + (5 + 7) * (6 + (8 * 6 + 9 + 8 * 6) + 7 + (2 + 2 + 9 * 3) * 4 + 4) * ((6 * 3 * 3 * 6 * 2) * (5 * 2 + 4) + (4 * 4 + 7 + 6 + 4 * 3) * 9) + 9
((5 + 2 + 6 * 3 * 7 * 2) * 7 * 3 * (8 * 4 * 2 * 6 * 5) * 8 * 7) + 8 * 8 * 9 * 6
4 + 9 + (9 + 8 * 9 + 5 + 3) * 7 + 6 * 9
7 + 9 * (5 * 5 + 9 * 2 * 3 * 5) * 5 * 9
((3 * 9 * 2 + 2 * 4) * 7) * 7 * 9
3 * 5 + 5 + (3 * 9 * 7 * 6 + 8 + (4 * 6 + 7 + 8 * 5)) + 8 * 4
9 * 9 + 5 + 2
(7 * 8 + 6 + 5) + 7
(5 * 2 + 2 + 7) * 2 * (8 + 4 + 9 * 6) * 2 * (8 + 6 * 3 * 3 + 9)
6 * 2 + 5 + (8 + 8 * 6 + 9)
3 + (2 + 7) + (4 + 4 * 4) + 5 * (9 * 4 * 4 * 3 + 5 + 8) * (5 * (8 * 9 + 9 * 7 * 4) + 3 * 9 * 2)
6 * 4
2 + 7 + 5 * (8 + 2 * 2 * 6 * 6 + (5 * 6 + 9 + 6 * 6))
7 * ((8 + 9) + (7 * 8 * 5 * 3 * 2) + 4) + 7 * 6
2 + (8 + 3 * 6 * (5 * 9 + 9 + 8 * 8 * 2) * 5 + 5) + 5 + 3
((5 + 5 + 2 * 8) + 6 + (3 * 9 * 3 * 9 + 8)) + ((2 + 4 + 7 * 9) + 9 * 5 + 2 + 9) * 5 * 5 + (9 + 4) * 9
6 + (7 + 7 + 4 * 5 + 5) * (3 + (5 * 5 + 4) + (6 * 2 * 9 * 3 + 4)) * 4
2 + 2 * (8 * 6 + 5 * 9 + 7 * 3) * 2
(2 * 7 * 2 * 2) * 4 * 4
(3 + 5 * (4 + 2 * 7 * 4 + 4 + 9)) + 7 + 7 * 8 + 2 * 5
(7 + 4 * 6 + 3 + (7 + 5 + 3 + 2 * 4) + (6 + 9)) + 6 * 8 * 7 + 4 * 9
4 + 2 + 8 * 3 + 5 + (6 * 5 + 7)
(6 + 9 + 9 + 8 * 3 + 2) + 8 * 7
5 * 9 + 7 + 5 * (2 * (9 * 5 * 2 * 9 + 9) * 5 * (7 * 5 * 2) * 9 * 6) + 4
(7 + 2) + 3
4 * 7 * 3 * (8 * 7 + 4) + 6 + (2 + 7 * 2 + (3 * 4 + 4 + 2 + 2) * 4)
8 + (9 * 9 + (9 * 9)) * 9 * 9 + 6 + 9
(2 * 8 * (2 * 6 + 2) * 9) * 3 + (4 * (9 + 6 * 9 + 6)) * 9 + 2 + 3
8 + (2 + 4 + 5 + 7 * 2) * 7 * 5 + 8 + 9
4 + 3 + (6 + (3 + 5 + 2 + 2 * 4) + (6 * 8 * 6 * 7 + 7 + 2) * 9 * 7 * 7)
((6 * 4 * 7) + 5 + 8 + 8 + 6) * 6 + 9 + 3
4 + 4 + (5 + 7 * 9 + 9 * (4 * 4 * 4 + 6 * 2 * 5))
5 + (9 * 3 + 2 + 9 + (3 + 9 * 4 * 7)) * (7 + 8) * 8 + 3 * (6 + 2 * 4 + 2 * 5)
7 + (9 * 4 * 3 * 4 + (7 + 3 * 4 + 8 * 5)) + 8 + 8 + 6
3 + 3 * 7 + (5 * 8) + (6 * (8 * 7)) + 7
6 * (2 * 9 * 9 + 6 + 3) * (5 * 9 * 7 + 4 * 6 + 7) + (4 + 4 * (4 * 3 + 5))
2 + 9 + 7 + (2 + 8 + 8) + 5 + 9
4 * 6 * (3 + 8 + 2 * 4) + 7 + 5 + (3 + 9 + 3 + 3 * 5)
8 + ((5 + 5) + 2 + 3)
(9 + 2 + 4 + 7) + (4 * 7 + 3 + 4) * 9 + 6
8 + 5 * (9 + 9 * 4) * 6 * 4 + (8 * 2 * 3 + 7 * 7)
8 + (2 + (3 * 5 + 7 * 3) * 8 * 9 * 2) * (6 + 5 * 2 + 9) + 3 * 8 + 7
3 * (5 * 6 + 3 * 7) + 6 + 8
6 + (5 * 9 + (4 + 9 + 9) * 6 * (8 + 5 * 7) * 5)
4 + (3 + 8) + 8 * 9 * (3 * (6 * 9 + 7 * 9 + 8 * 5)) + 6
(2 + 5) * 4 * 6 + ((2 * 4) * (9 * 6 + 8) + 7) * 9 * 6
7 * 4
4 * 8 * 9 + (6 * 4 * 6 + 5 + 7) * 3 * (4 * 5 + 9 + 6 * 5)
6 + (5 + 4 * 3 + 8 * 3 * (9 * 4 + 7 * 2)) * 6 + 5 + 5
4 + 8 * 7 * 2 + 2
((9 * 6 + 2 + 8 * 2 + 9) + 7 + 3 * 9 * 6 + 9) * 4
5 + 7 * 8 + (2 * 3 * (3 + 3) + 8 + 3)
3 * (8 * 2 + 3 + 3 + 6 + 2)
8 + 7 * 6 * 2 * (5 + 5 + 4) * 6
(3 * (8 * 2 * 9 * 8) * 6) + 3 * ((9 + 7 * 7 * 9 * 2) + 2 + 2 + 4 + 6 + 9) * 9 * (9 * 6 + 7 + 3 * 8) + (4 + (4 * 7 + 5))
3 * ((9 + 2 + 6) * 3 * 9 + 9 * (8 + 4))
((4 * 2) * (6 * 7 + 6 + 5 * 2 + 2) + 6) * 9 + (5 * 6 * 8 * 5 * (2 + 6 * 9 * 2) + 6) + 7 * 6 + (8 * 5 + 9 * (2 + 6) + 8)
(6 * (3 * 9 + 5 + 7 * 4) + (5 + 8 * 7 * 5 * 8) * 6 + 2) + 2 + 2 + 6 * 5 + (3 + 6 + 3 + 9 * 9 + 8)
6 * 4 * (4 + (9 + 9 * 3 * 3)) + 5 + 7
7 * (5 * 8 + 3) * 8 + 6
6 + 5 + 4 + (2 * (6 * 3 * 5 + 8 + 6) * 7 + 9 * 3) + (2 * 8 * 2 + 7)
7 + 9 * 8 * 8 * (2 + (5 * 7 + 2) + 7)
7 * (2 + 4 + 3 + 7 + (3 + 5) + (7 * 5 * 7 * 9 * 4)) + 2
8 * 2 * (8 * 8 + (3 + 2 * 9 + 6) * 3 + 8 + (2 + 2)) * (9 * 4 + (4 + 8 + 5 * 2 + 8 * 6) + 2 + 9 * 6) * 7 * (6 + 8 + (9 * 5 + 3))
3 * (4 * 8 * 3 + 3 * 6)
7 * 7 * (8 + 9 * 4 * 7) + 6 + 3 * 8
7 + (5 * (7 * 3 * 5))
(4 + 3 + 9) * 9 + 7 * ((7 * 8 + 9 * 3) + 8)
6 + 8 + (3 + 7 + 7 * 2 + 9) * 5 * 9
((7 + 3 + 7) + 9 + 9 * 6) + 6 + ((3 + 6 + 8 * 6 * 7) * 2 + (7 * 5 + 8 * 8 * 7 + 7))
(6 * 4 + 6) * 9 * 2 * 8 + 2
8 + 6 + 9
8 + 9 + 9 + 9 + (3 + 3 + 5 + 9 + 3 * 9)
(9 + 4 + (7 + 6 * 3 * 9 + 2 * 2) * 7 * 8 * 7) * 3 + 4 + 7
3 + 3 * 6 + (5 + 9) + (2 * 9 * 7 * 3)
3 * 6 + 8 * 2 + 3
8 * (2 + (2 * 2 * 9 * 4 * 8 + 4) * 2 * 2) * 3 * 5
(3 + 9 + 4 * 5 + (7 * 6 * 6) * 9) + ((7 * 9) * 2 + 3 + 9 + 4) + 5 + (2 * 5 * 2 * 5) * 6 + 7
4 + (6 + 3 * 5 * 7) + 9 * 2 * (3 + 2 * 4 * 2 + (2 * 8 * 9 + 4))
4 + 3 * (8 + 4)
7 * 6
9 + 7 * (9 * 4 + 4 + 9 + 5 * 2)
(7 * 7 + 8) + 3 * 5 * 3
(6 + 7 * 2 + 5) * (4 + 5 * 6 * 3 * (4 + 4 * 5 * 2 + 6 * 2) + 4) + 9 * 7
(8 + 7 + 5 + 8) + 2 * (7 + 4 * 8 + (2 + 2 + 5 + 4 * 5 + 4)) + 6 + (3 * 5) + 3
3 * 7 * (9 * (2 * 4 * 3 * 8) + 5 * 4 * 5 + 8) * 4 * 4
7 + 4 + (9 * 3 + 4 * 5 + 4)
((5 * 5 * 4) + 9) * (3 * 6 * 6 * (9 + 8 + 7 + 4 * 3)) + 6
(8 + 9 * 8 * 7 * 5 * 6) * 6
7 + 2 + 5 * (8 * 7) * 9 * 4
3 * 4 * 6 * 4 * 6 + 6
9 * 4 * 7 * 8 + (7 + 4 * (2 + 5) + (8 * 9 * 6 + 4 + 8) * 7)
(7 + 6 * 6 * 7 * 3) * (4 * 7 * 6) + 8 + 9 + 7
7 + 9 * (4 * 9 * 2 * (7 * 6 * 7) + 3) + ((6 * 8 * 3 * 6 * 9) * 4 * 3 + 2 * (6 + 3))
7 + (2 + (3 + 4 + 5 + 6 * 7 * 2) + 7 + 3 * (2 * 6 + 3 + 9 * 7 * 9))
(6 * 5 + 8) + (4 * (8 + 9 + 6) * (7 + 6)) * 3 + 9
3 * 8 * (3 + 3 + 5) * 8
2 * (6 + 3 * (6 + 6 * 2) + 8 * (4 * 4)) + 9 + (2 * 3 * 8 + 2) * 7 + 6
4 * 4 + 5 * 5 * 5 * 6
2 * 8 + (5 + 8 + 6)
5 * (6 * 7 * (6 + 8 + 9) * 8 + (3 + 8 + 2 + 2 + 7)) * 4 + 3 * 5
((4 + 9 + 3 * 3 + 9 * 3) * 2 + 2 + (5 + 6) + 2) + (3 * 2)
6 + 2 * (7 * (2 * 6 * 7) + 8) * 2 * 3 * 3
2 * ((8 + 2) + 5 + 3 + 4) * 7 * 7 * (3 + 6 * 7)
(9 + 5 + 6 * 5 + (5 * 9 * 2 * 4 + 9) + (8 + 3 * 2)) + 7
(4 + 2 + (3 + 7 + 3 * 3)) * 9 * (9 * 2 * 6 * 4 * 4 * (4 * 9))
4 + (9 + 6 * 8 + (6 + 6 * 2) + 3) * (5 * (2 * 3 * 5 * 5 * 5) * 5 * 7) + (5 * 6)
5 * 4 + 4 * 9
(3 + 7 * 7) * ((2 * 4 + 8 + 2) * (9 + 7 + 7 * 8) * 4 + (8 + 4 + 2 * 4 * 5) + 8 + 6) + (9 + 8 * 6 + 4 + 2 + 5) * 6
(4 + 7 * 3 + 7) + (5 * 9) * 6 + 7
5 * 3 * (6 * 6 * 5 + (5 * 8)) * 8
((3 * 7) * 5 * 3) + 3 * 3 + 6
(8 + 8 + 3) * (7 * 9 * (6 * 6) + 7 * 5 + 6) + 6
3 * (3 * 7) * 5 * ((4 + 5) * 4 + (2 * 5 * 7 * 4 * 3) + 5)
(3 + 6 + 4 + 8) + 9 + (7 * 5 * 7 * 3 + 5 * 6)
6 * (3 + (3 * 8 * 6 + 3 + 5 * 9))
(5 * 6) * 4 * 4 + (3 + 4) * 7
7 + 2 * 9
5 + ((4 + 7 + 5 * 4 * 5) + 5 * 3 + 8 + (3 + 6 * 6 + 4))
3 * 7 + (3 + 4 * 9 + (6 * 5 * 4) + 7)
7 + (8 + 7 + 8 + 5 * 6) * 3 * 3
8 + ((8 + 3 + 9) * (3 * 4 + 7 * 4 + 3)) * 9 + 9 + 8
6 + (7 * 7 + 8 + 2 * 4) * 6 * (8 + 8 * 7 + 5 + 5 * 6) * 2 + 9
4 * 2 + 4 * (8 + 6 * (3 * 9 * 6) * 8 * 8 + 7) * 5
((6 * 2 + 2 + 2 * 8) + 3 + 4 + 9 * 9 + 6) * 8 + 3
(4 + 2 * 5 * 3 + 5) * 7
5 + 8 * (2 + 9 * 5 + 4 + (3 + 4 * 6) + 3) + 4 + 7
4 + 5 * 3 * (9 * 7)
4 + (2 + (4 * 9 * 6) + 7 * 9 * 9 + 2) * (2 + 8 * (4 + 4 * 5 + 4 * 2)) * 3
((6 + 7 + 2 * 7 * 3) * 7 + (7 * 7 + 5 + 3 + 6) + 2) + (4 * 5 + 3 + (7 * 9 * 2 + 8 * 9) * 9) + 5 * 7
6 + 2 + (6 * 9) + 3
3 + 3 + ((7 * 6 + 8) * (6 + 5 * 5 + 8) + 5 + (7 + 9) + 4 + 4)
6 * 7 * 8 + (5 * 5 + 9 * 7 * 8 + 4) + 4
(6 + 4 * 8 * 6) * (4 * 6) + 7 + 8 * 8 * (9 + 4 + 4 * 6 + 7)
3 * 5 * (9 + 3) + 9 * 4 + 6
(5 + 4 + 5 + 6) * (5 + 5 * (4 * 7 * 4 + 7) * 2 + 3 + 8) + 9
6 * 6 + (9 * 2 * (2 + 8 + 3 + 7 * 2) * (6 + 2 * 2))
7 + 4 * (9 * 4 * 9 * 9) + 9 + (5 + 8 * (6 * 7 * 7 + 2 * 5 + 8)) * 2
((8 * 2) * 8 + 8) * 8 + 3
8 + 8 + ((6 * 2 + 5) + (3 + 9 * 3 * 6)) * 6 * 8
3 + (2 * 3 * 2 * 6 + 3) + 3
(4 + (7 * 9 + 9 + 2) + (6 + 4 * 8) + 2) + 6 * (7 + 8 + 6 + 2 * 3 + 8)
5 * 6 + (3 * (7 + 7 * 6)) * 6 * 7
(6 + 6 + 6 * 4) * 3 + 3 + ((4 + 8 + 8 + 2 + 3) + 7 * 4)
6 * 8 * (5 * 8 + 9 + 9 + 9 + 4) * 2 * 2 + 9
9 + 8 + (6 * 7 + 4 * 6)
4 * 9 + (3 + (7 + 8 + 5) + 9 * 4)
2 + (4 + 7) + 3 * 3
2 + 4 + ((5 * 7 + 6 + 7 * 3) + 7 * (8 + 6 * 4 + 7 + 5) + 3 * 4 + 9) * 8 + 6 * 7
5 + 2 * (5 * 3 * 7 + (3 + 7) + (6 + 4))
(3 * 9 * 7) + ((6 + 7) + (5 + 2 * 4 * 7 * 3)) * 8 + 8
9 * 5 * 4 * (5 + 4 + 5) + 2
(6 * 9) + ((7 + 2 * 6 + 5 * 5 + 5) * 4 + 7) + (5 + 7 + 5 + 3 * 9 * 5)
(3 * 3 * 2 * 6 + (6 + 6 * 4 * 7 * 5) + 5) + 8 + 3 + 5 * 5
((9 * 3) * (5 + 9 + 2) + 5) + 4 + 3 + (8 + 7) + 7 + 9
(4 * 9 * 2 * (9 * 9 * 2 * 6 + 4) * (8 + 9 * 4 * 6 * 3) * (3 + 6 + 4)) * 5 * (2 * 2 * 2 * 7 + 5) + 2 + 5
(6 + 4 * 9 * 6 + (5 + 3 + 9 * 6)) + 5 * 3
3 * (8 + 7 * 8 + 6 * 7) * (5 + 2 + 6 + (2 + 7 + 5)) + 3 * 5
4 + 8 * 6 * (8 * 9) + 9 + (3 + 9 + 2 + 2 + 3 * 4)
5 + (6 + 5 + 2 + 8) + 6 + 4 * 6
9 + ((7 * 9 * 7 * 9 * 5) * 8) * 9
3 + 6 + (9 + 3 + 7 + 7 * 8) + 5 + 7 * (8 * 5 * 8 * (4 * 8 * 8 * 9 * 9 * 7))
5 * 2 * 4 + 7 + 7
3 * (8 * (4 * 8) + (5 * 5 * 8 + 9 * 9 * 3) + 9 + 6 * 5) * (4 * (5 + 7 * 6 + 6 * 3) + 6 + 3) + 4 * 4 + (6 + 3 * 7 + 3)
4 + 4 + 2 + 3 * ((5 * 6 * 7) * 8 + 6 * (9 * 8 * 3 + 8 * 3 + 6) * 8)
8 + ((5 + 2 + 9 * 3 + 8) + 3 * (6 + 9 * 5 + 6) * 8 + 2)
(8 * 6 + 3 + 5) * 4 + 4 * 7 + (3 * 5) + 6
3 + (8 + (4 + 9) * (3 + 5 + 9 * 2 + 6) + 9 + 8) * 4 + 6 * 9
2 + 3 + 8 + ((8 * 6 * 5 * 2) + 9 * (4 + 5 + 6 + 7 * 3 * 7) + 7 * 3) * 2
(3 * 6 + 3) + 6 * 5 + 7 + 2 + 4
6 + (5 + 6 * 2 + 3) * 7
4 + (9 + (6 * 7) * 3 * 9 + 6)
9 * 4 * 6 * 5 + (7 * 6 * 9 * (8 * 3 * 6 * 4 + 2 + 7) * (8 * 5 + 3 * 6 + 3))
(9 + (4 * 3 + 8 + 2 * 3 * 7) + 7 + (6 * 9 + 9 + 7 * 5 * 6) + 3 + 9) * 6 * 3 + 2 + 7
5 * 2 * 9 + (4 * 8 * 5 + (2 + 3 + 3 + 8 + 6) + 8) + 3 + 4
4 + 5 * (5 + 4 * 9 * (9 + 6 * 4)) + 3 + 6 + 4
9 + 4 * (5 * 8 + (8 + 7 + 4 * 8 + 2)) + 5 + 2 * 6
2 * 8 + (4 * (2 + 2 * 3 + 2 + 2) * (4 + 9 + 5 * 7) * 2 * (5 + 7)) + ((8 * 8) * (2 + 9 + 6 + 8 * 6)) + 6 + (7 + (5 + 2 + 9 * 6) + 7 * 7 * 6 * 5)
4 * 8 + 8 + (5 * 3 + 2 + 2 * 7 * 3)
7 * 5 + 2 * (8 * 7 * 6 + (8 * 7) * 4 + 2) * 7
2 + 6 * ((5 + 7 + 2 + 7 + 8) * 9 + (8 + 4) * 2)
(2 + 7 * 3 * (9 * 8 * 3 * 2 + 7 * 2) * 5 * 9) * 7 * 9 * 5 * 4 + 8
6 * 6 * 4 + (9 * 7 * (2 * 5 * 4 + 8 + 2) + 4 + 3) * 3
2 + 7 * 4 * (4 * (9 + 5 * 6)) + 7
6 * (2 * 3 * 9 * 2 * 4 + (7 + 7 + 5 * 6 * 3)) * 3 * (8 + 6) + (8 * 9 * (5 + 6 + 4 + 3 * 3 * 4) * 2) * 5
(8 + (8 + 6)) * 6 + 2
9 + ((8 + 5 * 3 * 2) + 4 * 6) + 8 + 4 * 5 + 3
4 * 3 + 4 + (6 * 8 + (6 + 3 * 9) + 8 + (3 + 6 * 3) + (7 * 8 + 9)) * 4 + 7
7 + (8 + 6) + 8 + 6 * 9
(5 + (4 + 6) + 8 * 9 + (8 * 6 + 7 * 4 + 2) * 8) * 6 * 9
4 * 7 + (7 * 8 + 6 * 7 * 7) * (6 + 7 + 8 + (2 * 7 * 2 * 3 * 3))
((3 + 7 + 3 + 6 + 6 * 4) * (3 * 6 * 7 + 5 + 3 + 3) + 7 + 2 * (4 * 4 * 6 + 5) * 6) + 8
((2 * 3) + (2 + 2 + 3) + (2 + 5 * 4 * 4 * 2 + 5) * 4 * 6 + 2) + 5 + 5 + 5
8 * (5 * 9 + 5 + 4 + 3) * ((4 * 5 + 5 * 8 + 9 + 2) * (7 + 4 * 4 + 6 + 8)) + 6 + 9
(3 * 3 + 3 + (4 * 9) * 5) + ((9 + 3 * 6 * 9 * 4 * 5) * 8 + 3 * 5 * 3 + 7) + 4 * 3
(2 * 6 * 4 * 3 * 7) * (2 + (8 + 5 + 4 + 2 * 8) + (2 * 8 + 9 * 6 + 2 + 8) + 9 + 2 + 2) * 5 * (5 + 3 + 4)
9 * (8 + 6) + 7
5 + (8 + (2 + 2 + 3 * 2 + 8 + 3)) * 9 + (8 * 8 * 2) * (3 + 8 + 6 * 7 * 9 + 2) * (4 + 7 * 9 + 8)
((9 * 9 * 9) + (7 * 7 * 6)) + (5 * (8 * 4 * 4) + 3 + 2 + 5 + 6) + (5 * 9 * 8 + 2 * 2 * 5) + 8 * (7 * 2 + 8 + 3 * 3 + 3)
(4 * 8 * 5 * 7 * 6) + (2 + 2 * (5 * 7 + 8 * 2 * 3 + 5) * 9 * 9 * 5) * 2 + 4 * 6 + 9
(4 * 6) * (5 * 2 + (9 + 5)) + 7 + 8 * 2 * 3
((2 * 3 * 6 * 4 * 6 * 2) + 4 + 5 + (8 + 8 + 5) + 6 + (7 * 6 + 9 + 3 * 6)) + 2 * 9
5 + (2 + 2 * 5 * (3 + 6 * 6 * 2) * 5 * (7 + 4 + 3 * 2)) + 6 * 9 * 8 * (7 * 6 * 8 + (9 * 2 + 3 + 2 * 6))
(7 * 9 * 3 * 4 * 8) + 7 * 3 * (8 * 4 * (7 + 7 * 4 + 8 * 3) + (7 + 6 * 4) * 9 * 7)
9 * (9 + 3 * (7 + 4 * 9) + (5 + 9 * 2 + 5)) * (5 + 8 * 8)
8 * (6 + (7 + 8) + 8 * 4 + 7) + 7
2 + ((6 * 8 + 5 + 2 + 8) * 5 + 8) + 9
(6 * 2 + 7 * 8 + 4 + 7) + 5 * 7 + 7 * 3
2 * 4 * 4 + (4 * 6 + (9 * 5 * 5 * 5 + 8) * 6)
4 * 7 * 9 + ((2 + 3) * 2 + 2 + 6 * (7 * 2)) + 7
2 + (9 * 6 + (3 * 3) + 3 + 4) * 2 * (8 + 7 * 6 + 4 + 5 * 3) + 8 * 2
7 * 7 * (4 * 6 * 5) + 8 + (2 * 2 + 4 + 2 * 8) * 3
8 * 6 + ((6 + 2 + 3 * 4) + 8)
5 + (4 + 9 * 3) * 4 * 9 + ((8 + 3 + 2 * 5 * 9 + 9) * 5 + 3 + (8 + 9) * 4 + 6) + 8
8 * 6 * 3 * (3 + 2 * 5) + (6 + 8 * 2 + 4) * 8
(2 * (2 * 6 + 3) * 6 + 5 + 7) + 9 + 4
7 * 2 * 3 * (3 + (7 + 2) + 4 * 5)
2 + 9 * 8 + (4 + 4 + 6 * 2) * (7 * 4) + 8
(5 * 4 + 6) + (2 + 5 * 3) * 7 + 5 * 2 + 5
7 * 5 + 9 * 8 * 3 + (5 + 3)
9 * (3 * 8 + 6 * 4 * (3 + 6 * 7 + 5)) + 8 * 3
6 * 4 * 9 * 6 + ((5 + 4) * (3 + 8 + 5 + 6))
(8 + 8) + 2 + 7 + 5 + 9
9 * 3 + 3 + (2 * 4 * (9 + 8 * 2 + 9 + 3) + 2 + 9 * 9)
4 * (2 * 3 + 9 + (2 + 3) * (6 + 2)) * 2 + 5 + (4 + 2 * 5 + 8 + (8 * 5 * 4) * 2) + 5
3 * 5 + 8 + ((9 + 4 + 8 + 4 * 5 + 7) + 6)
4 * (2 + 3) + (8 * 6 + 6 + 7 + (9 * 3 * 3 * 8 * 3)) + 5 * 5 * (2 + 7 * (6 + 5 * 6 + 3 + 4) + 5 + 9 + 2)
(9 * 4 + 2 + 6 + (4 + 8 * 8 + 9 + 3 * 5) + (8 * 8 * 4)) + ((8 + 2 * 7 + 9 * 4 * 7) * 7 + 6 + 3 * 2 + 8) * 2 * 9 * (8 + (6 + 8 + 8 * 5 * 9 * 8)) * 5
6 + 2 + (9 * 6 * 8) + (3 * 2 + (6 * 3 + 2 * 9))
5 * 3 * 3 * 6 * (2 * 5 * 9 * 8) * 9
2 * 8 + 2 * 3 + (6 * 4 * 3 + (6 + 3 * 7 + 2 * 8) * 4)
9 * (7 * 4 * 4 * 5 + 9 + 5) * 4 + 2 * (2 + 8)
7 * 5 + 6 + 4 + 5 + 3
(9 + 4 + 7 + 2) * 4 + 2 * 4 + (3 + 6 * 9 * (2 + 6 + 2 + 8) + 3 + 9) * 7
7 * 9 + (7 + 3) + 8 * (4 + 2 + 5) + 7
8 + 4 + (3 * 9 + 9 + (8 + 5 + 4 + 6 * 4) * 4 + 9)
(4 + 6 * (3 + 9 * 6 * 5 * 9 + 4) * (2 * 9 * 2 * 7) * (7 * 6 + 9 + 3)) + 6 * (6 + (7 * 8 * 3 + 4 * 6))
((7 * 9 * 4 * 4) * 8 * 7 + 8 + 9 + 5) + 9
7 * ((7 * 8 * 3) * 2 * (8 * 7 + 2) + (3 + 2 * 6 * 9) * 5 * 8) * 4
(6 * 9) + (7 + 6 + (5 + 9 + 6 + 7 + 4 * 6))
3 * 6 + 8 + 7 + (6 + (2 + 9 + 4 + 3))
(9 * 5 + 6 * 5 * 9 * (5 + 2 * 2 + 3)) * (7 + 3 + 2 + 4 * 3) + (6 * 3 * 7 + (3 + 6) * 3) * (6 + 7) + 7
5 + 3 + 6 * ((9 + 6 + 8 + 6 * 5) + 6 * 6 + 5 + 3 + (5 * 9))
9 + (5 + (8 * 8 * 8 * 5 + 8 * 9) * 5) * 7 + 8
(7 + 2 * 7 * 6) + 6 + 9 + 7
7 + 8 + ((5 * 6 * 2) * (4 * 5 * 7 + 8 * 8)) * (9 * 5 + 8 * (5 * 3 * 3 * 5 + 3 * 3) * 9 * 9) * 9 * 6
((8 * 5 + 6 * 6) + 2 + (4 + 2 * 2 * 9 * 7 * 7)) * (7 + 9) + (9 + 3 + 7 * 8 + 4) * 5"""

In [22]:
import re
def solve(line):
    while '(' in line:
        print('line', line)
        line = re.sub(r'\(([^()]+)\)', doExpr, line)
    print('lastDoInner', line)
    return doInner(line)

def doExpr(match):
    inner = match.group(1)
    return doInner(inner)
PART = 1
def doInner(inner):
    # part 1:
    if PART==1:
        while '+' in inner or '*' in inner:
            print('   inner ?:', inner)
            inner = re.sub('^(\d+)\s*\+\s*(\d+)', lambda m: str(int(m.group(1)) + int(m.group(2))), inner)
            inner = re.sub('^(\d+)\s*\*\s*(\d+)', lambda m: str(int(m.group(1)) * int(m.group(2))), inner)
    if PART==2:
        while '+' in inner:
            print('   inner +:', inner)
            inner = re.sub('(\d+)\s*\+\s*(\d+)', lambda m: str(int(m.group(1)) + int(m.group(2))), inner)
        while '*' in inner:
            print('   inner *:', inner)
            inner = re.sub('(\d+)\s*\*\s*(\d+)', lambda m: str(int(m.group(1)) * int(m.group(2))), inner)
    print('inner', inner)
    return inner

for s in samples:
    print(s, solve(s[0]))

line (10 + 4 * 2 + 1)
   inner ?: 10 + 4 * 2 + 1
   inner ?: 28 + 1
inner 29
lastDoInner 29
inner 29
('(10 + 4 * 2 + 1)', 29, 42) 29
line ((10 + 4) * 2 + 1)
   inner ?: 10 + 4
inner 14
line (14 * 2 + 1)
   inner ?: 14 * 2 + 1
   inner ?: 28 + 1
inner 29
lastDoInner 29
inner 29
('((10 + 4) * 2 + 1)', 29, 42) 29
line ((10 + 4 * 2) + 1)
   inner ?: 10 + 4 * 2
inner 28
line (28 + 1)
   inner ?: 28 + 1
inner 29
lastDoInner 29
inner 29
('((10 + 4 * 2) + 1)', 29, 29) 29
lastDoInner 10 + 4
   inner ?: 10 + 4
inner 14
('10 + 4', 14, 14) 14
lastDoInner 1 + 2 * 3 + 4 * 5 + 6
   inner ?: 1 + 2 * 3 + 4 * 5 + 6
   inner ?: 9 + 4 * 5 + 6
   inner ?: 65 + 6
inner 71
('1 + 2 * 3 + 4 * 5 + 6', 71, 231) 71
line 2 * 3 + (4 * 5)
   inner ?: 4 * 5
inner 20
lastDoInner 2 * 3 + 20
   inner ?: 2 * 3 + 20
   inner ?: 6 + 20
inner 26
('2 * 3 + (4 * 5)', 26, 46) 26
line 5 + (8 * 3 + 9 + 3 * 4 * 3)
   inner ?: 8 * 3 + 9 + 3 * 4 * 3
   inner ?: 24 + 9 + 3 * 4 * 3
   inner ?: 33 + 3 * 4 * 3
   inner ?: 144 * 3
inner